In [1]:
# Se importa los paquetes necesarios
import pandas as pd
import numpy as np
import requests
import zipfile, os, sys
import json

# Carga del Dataset original

In [2]:
# Enlace del archivo
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/WorldBank%20ESG/esgdata_download-2025-09-22.csv"

# Se carga el dataset en un DataFrame pandas
ESG_df = pd.read_csv(url)

# Supprimer colonnes inutiles
ESG_df.drop(columns=['Country Code', "Indicator Code"], inplace=True)
ESG_df.sort_values(by=['Indicator Name'], inplace=True)
ESG_df

,Country Name,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
1346,Cameroon,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.8000,23.6000,24.3000,25.3000,26.4000,27.4000,28.6000,29.4000,NaN,NaN
1453,Qatar,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,NaN,NaN
1452,Paraguay,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,65.5000,66.4000,67.2000,67.7000,68.1000,68.3000,68.6000,68.5000,NaN,NaN
1451,Portugal,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,NaN,NaN
1450,"Korea, Dem. People's Rep.",Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.8000,9.3000,10.0000,10.6000,11.3000,12.3000,13.2000,13.9000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33272,Guinea-Bissau,Voice and Accountability: Estimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.7494,-0.7342,-0.7980,-0.6834,-0.4819,-0.4556,-0.2506,-0.4172,-0.4369,NaN
33273,Equatorial Guinea,Voice and Accountability: Estimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.9994,-1.9479,-1.9712,-1.8735,-1.8790,-1.8105,-1.7247,-1.7043,-1.6769,NaN
33274,Greece,Voice and Accountability: Estimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.6516,0.6689,0.6587,0.7418,0.8107,0.9629,0.9453,0.9569,1.0122,NaN
33263,Fiji,Voice and Accountability: Estimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0278,0.0287,0.1239,0.1459,0.0224,0.0561,-0.1721,-0.1440,0.1861,NaN


In [3]:
ESG_df['Country Name'].nunique()

214

In [4]:
ESG_df['Indicator Name'].nunique()

191

On est donc censé avoir 214 * 191 lignes = 40874 il en manque

# Limpezia del Dataset original y Utilidad


In [5]:
# Certaines lignes sont dupliquées
ESG_df = ESG_df.drop_duplicates()
ESG_df

,Country Name,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
1346,Cameroon,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.8000,23.6000,24.3000,25.3000,26.4000,27.4000,28.6000,29.4000,NaN,NaN
1453,Qatar,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,NaN,NaN
1452,Paraguay,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,65.5000,66.4000,67.2000,67.7000,68.1000,68.3000,68.6000,68.5000,NaN,NaN
1451,Portugal,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,NaN,NaN
1450,"Korea, Dem. People's Rep.",Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.8000,9.3000,10.0000,10.6000,11.3000,12.3000,13.2000,13.9000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33272,Guinea-Bissau,Voice and Accountability: Estimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.7494,-0.7342,-0.7980,-0.6834,-0.4819,-0.4556,-0.2506,-0.4172,-0.4369,NaN
33273,Equatorial Guinea,Voice and Accountability: Estimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.9994,-1.9479,-1.9712,-1.8735,-1.8790,-1.8105,-1.7247,-1.7043,-1.6769,NaN
33274,Greece,Voice and Accountability: Estimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.6516,0.6689,0.6587,0.7418,0.8107,0.9629,0.9453,0.9569,1.0122,NaN
33263,Fiji,Voice and Accountability: Estimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0278,0.0287,0.1239,0.1459,0.0224,0.0561,-0.1721,-0.1440,0.1861,NaN


In [6]:
# Certains indicateurs ne nous intéressent pas on les supprime
ind_to_delete = [
    'Carbon intensity of GDP (kg CO2e per constant 2021 US$ of GDP)',
    'Access to electricity, rural (% of rural population)',
    'Carbon dioxide (CO2) emissions (total) excluding LULUCF (% change from 1990)',
    'Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita)',
    'Carbon dioxide (CO2) emissions (total) excluding LULUCF (Mt CO2e)',
    'Carbon dioxide (CO2) emissions from Agriculture (Mt CO2e)',
    'Carbon dioxide (CO2) emissions from Building (Energy) (Mt CO2e)',
    'Carbon dioxide (CO2) emissions from Fugitive Emissions (Energy) (Mt CO2e)',
    'Carbon dioxide (CO2) emissions from Industrial Combustion (Energy) (Mt CO2e)',
    'Carbon dioxide (CO2) emissions from Industrial Processes (Mt CO2e)',
    'Carbon dioxide (CO2) emissions from Power Industry (Energy) (Mt CO2e)',
    'Carbon dioxide (CO2) emissions from Transport (Energy) (Mt CO2e)',
    'Carbon dioxide (CO2) emissions from Waste (Mt CO2e)',
    'Carbon dioxide (CO2) net fluxes from LULUCF - Deforestation (Mt CO2e)',
    'Carbon dioxide (CO2) net fluxes from LULUCF - Forest Land (Mt CO2e)',
    'Carbon dioxide (CO2) net fluxes from LULUCF - Organic Soil (Mt CO2e)',
    'Carbon dioxide (CO2) net fluxes from LULUCF - Other Land (Mt CO2e)',
    'Carbon dioxide (CO2) net fluxes from LULUCF - Total excluding non-tropical fires (Mt CO2e)',
    'Domestic comprehensive wealth index (real chained 2019 US$)',
    'Foreign liabilities (real chained 2019 US$)',
    'Fluorinated greenhouse gases (F-gases) emissions from Industrial Processes (Mt CO2e)',
    'GDP (current US$)',
    'Human capital (real chained 2019 US$)',
    'Human capital index (HCI) (scale 0-1)',
    'Human capital, female (real chained 2019 US$)',
    'Human capital, male (real chained 2019 US$)',
    'Methane (CH4) emissions (total) excluding LULUCF (% change from 1990)',
    'Methane (CH4) emissions (total) excluding LULUCF (Mt CO2e)',
    'Methane (CH4) emissions from Agriculture (Mt CO2e)',
    'Methane (CH4) emissions from Building (Energy) (Mt CO2e)',
    'Methane (CH4) emissions from Fugitive Emissions (Energy) (Mt CO2e)',
    'Methane (CH4) emissions from Industrial Combustion (Energy) (Mt CO2e)',
    'Methane (CH4) emissions from Industrial Processes (Mt CO2e)',
    'Methane (CH4) emissions from Power Industry (Energy) (Mt CO2e)',
    'Methane (CH4) emissions from Transport (Energy) (Mt CO2e)',
    'Methane (CH4) emissions from Waste (Mt CO2e)',
    'National comprehensive wealth index (real chained 2019 US$)',
    'Nitrous oxide (N2O) emissions (total) excluding LULUCF (% change from 1990)',
    'Nitrous oxide (N2O) emissions (total) excluding LULUCF (Mt CO2e)',
    'Nitrous oxide (N2O) emissions from Agriculture (Mt CO2e)',
    'Nitrous oxide (N2O) emissions from Building (Energy) (Mt CO2e)',
    'Nitrous oxide (N2O) emissions from Fugitive Emissions (Energy) (Mt CO2e)',
    'Nitrous oxide (N2O) emissions from Industrial Combustion (Energy) (Mt CO2e)',
    'Nitrous oxide (N2O) emissions from Industrial Processes (Mt CO2e)',
    'Nitrous oxide (N2O) emissions from Power Industry (Energy) (Mt CO2e)',
    'Nitrous oxide (N2O) emissions from Transport (Energy) (Mt CO2e)',
    'Nitrous oxide (N2O) emissions from Waste (Mt CO2e)',
    'Nonrenewable natural capital per capita, coal (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, metals and minerals, sub-index, total (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, metals and minerals: bauxite (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, metals and minerals: cobalt (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, metals and minerals: copper (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, metals and minerals: gold (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, metals and minerals: iron ore (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, metals and minerals: lead (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, metals and minerals: lithium (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, metals and minerals: molybdenum (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, metals and minerals: nickel (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, metals and minerals: phosphate (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, metals and minerals: silver (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, metals and minerals: tin (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, metals and minerals: zinc (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, natural gas (real chained 2019 US$)',
    'Nonrenewable natural capital per capita, oil (real chained 2019 US$)',
    'Nonrenewable natural capital, coal (real chained 2019 US$)',
    'Nonrenewable natural capital, metals and minerals, sub-index, total (real chained 2019 US$)',
    'Nonrenewable natural capital, metals and minerals: bauxite (real chained 2019 US$)',
    'Nonrenewable natural capital, metals and minerals: cobalt (real chained 2019 US$)',
    'Nonrenewable natural capital, metals and minerals: copper (real chained 2019 US$)',
    'Nonrenewable natural capital, metals and minerals: gold (real chained 2019 US$)',
    'Nonrenewable natural capital, metals and minerals: iron ore (real chained 2019 US$)',
    'Nonrenewable natural capital, metals and minerals: lead (real chained 2019 US$)',
    'Nonrenewable natural capital, metals and minerals: lithium (real chained 2019 US$)',
    'Nonrenewable natural capital, metals and minerals: molybdenum (real chained 2019 US$)',
    'Nonrenewable natural capital, metals and minerals: nickel (real chained 2019 US$)',
    'Nonrenewable natural capital, metals and minerals: phosphate (real chained 2019 US$)',
    'Nonrenewable natural capital, metals and minerals: silver (real chained 2019 US$)',
    'Nonrenewable natural capital, metals and minerals: tin (real chained 2019 US$)',
    'Nonrenewable natural capital, metals and minerals: zinc (real chained 2019 US$)',
    'Nonrenewable natural capital, natural gas (real chained 2019 US$)',
    'Nonrenewable natural capital, oil (real chained 2019 US$)',
    'Nonrenewable natural capital, total (real chained 2019 US$)',
    'Produced capital (real chained 2019 US$)',
    'Renewable natural capital per capita, agricultural land (real chained 2019 US$)',
    'Renewable natural capital per capita, fisheries (real chained 2019 US$)',
    'Renewable natural capital per capita, forest recreation, hunting and fishing services (real chained 2019 US$)',
    'Renewable natural capital per capita, forest water ecosystem services (real chained 2019 US$)',
    'Renewable natural capital per capita, hydropower energy (real chained 2019 US$)',
    'Renewable natural capital per capita, mangroves (real chained 2019 US$)',
    'Renewable natural capital per capita, nonwood forest protection ecosystem services (real chained 2019 US$)',
    'Renewable natural capital per capita, timber (real chained 2019 US$)',
    'Renewable natural capital, agricultural land (real chained 2019 US$)',
    'Renewable natural capital, fisheries (real chained 2019 US$)',
    'Renewable natural capital, forest recreation, hunting and fishing services (real chained 2019 US$)',
    'Renewable natural capital, forest water ecosystem services (real chained 2019 US$)',
    'Renewable natural capital, hydropower energy (real chained 2019 US$)',
    'Renewable natural capital, mangroves (real chained 2019 US$)',
    'Renewable natural capital, nonwood forest protection ecosystem services (real chained 2019 US$)',
    'Renewable natural capital, timber (real chained 2019 US$)',
    'Renewable natural capital, total (real chained 2019 US$)',
    'Total greenhouse gas emissions excluding LULUCF (% change from 1990)',
    'Total greenhouse gas emissions excluding LULUCF (Mt CO2e)',
    'Total greenhouse gas emissions including LULUCF (Mt CO2e)',

    ]
print(len(ind_to_delete))
ESG_df = ESG_df[~ESG_df['Indicator Name'].isin(ind_to_delete)]
ESG_df['Indicator Name'].nunique()

103


88

In [7]:
ESG_df

,Country Name,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
1346,Cameroon,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.8000,23.6000,24.3000,25.3000,26.4000,27.4000,28.6000,29.4000,NaN,NaN
1453,Qatar,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,NaN,NaN
1452,Paraguay,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,65.5000,66.4000,67.2000,67.7000,68.1000,68.3000,68.6000,68.5000,NaN,NaN
1451,Portugal,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,NaN,NaN
1450,"Korea, Dem. People's Rep.",Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.8000,9.3000,10.0000,10.6000,11.3000,12.3000,13.2000,13.9000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33272,Guinea-Bissau,Voice and Accountability: Estimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.7494,-0.7342,-0.7980,-0.6834,-0.4819,-0.4556,-0.2506,-0.4172,-0.4369,NaN
33273,Equatorial Guinea,Voice and Accountability: Estimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.9994,-1.9479,-1.9712,-1.8735,-1.8790,-1.8105,-1.7247,-1.7043,-1.6769,NaN
33274,Greece,Voice and Accountability: Estimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.6516,0.6689,0.6587,0.7418,0.8107,0.9629,0.9453,0.9569,1.0122,NaN
33263,Fiji,Voice and Accountability: Estimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0278,0.0287,0.1239,0.1459,0.0224,0.0561,-0.1721,-0.1440,0.1861,NaN


Il manque toujours des lignes pour certains indicateurs 214*88 = 18832

In [8]:
# Ajouter une ligne pour les indicateurs manquants par pays

# Identifier toutes les combinaisons possibles
all_countries = ESG_df["Country Name"].unique()
all_indicators = ESG_df["Indicator Name"].unique()

# Créer un DataFrame complet avec toutes les combinaisons possibles
full_index = pd.MultiIndex.from_product(
    [all_countries, all_indicators],
    names=["Country Name", "Indicator Name"]
)

# Réindexer ton DataFrame existant sur cet index complet
ESG_df = (
    ESG_df.set_index(["Country Name", "Indicator Name"])
      .reindex(full_index)
      .reset_index()
)

In [9]:
ESG_df

,Country Name,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Cameroon,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.8000,23.6000,24.3000,25.3000,26.4000,27.4000,28.6000,29.4000,NaN,NaN
1,Cameroon,Access to electricity (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,58.6000,59.8000,60.9000,62.2000,63.2000,64.3000,65.4000,71.0000,72.0000,NaN
2,Cameroon,"Adjusted savings, natural resources depletion ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.4405,5.2334,5.8511,5.6127,5.0737,4.0527,4.9676,NaN,NaN,NaN
3,Cameroon,"Adjusted savings, net forest depletion (% of GNI)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.6341,3.7438,3.8193,3.0194,2.6998,2.7600,2.6397,NaN,NaN,NaN
4,Cameroon,Agricultural land (% of land area),NaN,15.8871,16.014,16.1198,16.2467,16.3737,16.5006,16.6487,...,20.6258,20.6258,20.6258,20.6258,20.6258,20.6258,20.6258,20.6258,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18827,American Samoa,Total greenhouse gas emissions per capita excl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.1589,0.1608,0.1628,0.1650,0.1693,0.1708,0.1727,0.1758,0.1789,NaN
18828,American Samoa,Tree Cover Loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18829,American Samoa,"Unemployment, total (% of total labor force) (...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18830,American Samoa,Unmet need for contraception (% of married wom...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Concernant le mapping certains choix ont été fait :
- "China, mainland": "China", pour celui là c'est normal world bank exclut déjà taiwan et hong kong des données
- "Macedonia, FYR": "North Macedonia", Simple changement de nom en 2019 mais même territoire
- "Swaziland": "Eswatini", Changement de nom en 2018
- "Congo" : "Congo, Rep.", Correspondance dans l'ensemble des datasets (dans le cas d'une erreur importante à vérifier)
- "Korea": "Korea, Rep.", Correspondance dans l'ensemble des datasets (dans le cas d'une erreur importante à vérifier)
- "Micronesia": "Micronesia, Fed. Sts.", géré au cas par cas
- "Ethiopia PDR": "Ethiopia", Changement de nom mais même territoire, cessesion de l'erythnée en 1993 mais pas relationé
- Ignoré tous les pays qui ont fait scission ou avant réunification

In [10]:
# Utilitaire

# Liste des agrégats de pays du dataset
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Utilidad/group_of_countries.txt"
response = requests.get(url)
group_of_countries = [line.strip() for line in response.text.splitlines() if line.strip()]

# Dictionnaire de mapping des noms de pays
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Utilidad/country_name_map.json"
response = requests.get(url)
country_name_map = json.loads(response.text)

# Liste des pays ou agrégats à ignorer dans les nouvelles données
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Utilidad/countries_to_ignore.txt"
response = requests.get(url)
countries_to_ignore = [line.strip() for line in response.text.splitlines() if line.strip()]

In [11]:
# Liste des pays avec agrégats
list_countries_with_group = ESG_df['Country Name'].unique()

nb_countries_init = len(list_countries_with_group)

# On enlève les agrégats de pays
ESG_df = ESG_df[~ESG_df['Country Name'].isin(group_of_countries)]

nb_countries = len(ESG_df['Country Name'].unique())

In [12]:
# Utilitaire

# Nombre de colonnes années
N_years = len(ESG_df.columns)-2

# Nombre de valeurs du Dataset (NaN inclut)
nb_val_dataset = ESG_df.drop(columns=["Country Name", "Indicator Name"]).size

# Nombre de valeurs par indicateur (NaN inclut)
nb_val_ind = nb_countries*N_years

# Liste des pays du DataFrame avec groupes + pays à ignorer
list_countries_complete = list(list_countries_with_group) + list(countries_to_ignore)

# Dataframe version long
ESG_df_long = pd.melt(ESG_df, id_vars=['Country Name', 'Indicator Name'], var_name='Year', value_name='Value_ref')

# Dictionnaire pour les nouvelles données
new_datas = {}

In [13]:
# Calculer le nombre et le % de valeurs manquantes du Dataset
def missing_per():
  nb_missing = ESG_df.drop(columns=["Country Name", "Indicator Name"]).isna().sum().sum()
  percentage = np.round(nb_missing/nb_val_dataset*100,2)
  return nb_missing, percentage

In [14]:
# Calculer et afficher le % de valeurs manquantes d'un Indicateur
def missing_per_ind(ind):
  nb_missing = ESG_df[ESG_df['Indicator Name'] == ind].drop(columns=["Country Name", "Indicator Name"]).isna().sum().sum()
  percentage = np.round(nb_missing/nb_val_ind*100,2)
  return nb_missing, percentage

In [15]:
# Calculer la différence relative moyenne d'un indicateur entre ESG et le nouveau dataset
def diff_moy_rel(ind):

  columns = ESG_df_long.columns.drop(["Indicator Name"])
  df_ref = ESG_df_long[ESG_df_long['Indicator Name'] == ind][columns]

  df_new = new_datas[ind].copy()
  df_new["Year"] = df_new["Year"].astype(str)
  df_new[ind] = df_new[ind].astype(float)
  df_new.rename(columns={ind: "Value_comp"}, inplace=True)

  merged = pd.merge(
    df_ref, df_new,
    on=["Country Name", "Year"],
    how="inner"   # seulement les couples communs
  )

  ref = merged["Value_ref"]
  comp = merged["Value_comp"]

  # Cas : les deux valeurs sont 0 → erreur relative = 0
  both_zero = (ref == 0) & (comp == 0)

  # Cas : ref = 0 et comp ≠ 0 → NaN
  ref_zero = (ref == 0) & (comp != 0)

  # Cas normal
  normal = ~both_zero & ~ref_zero

  rel_error = pd.Series(np.nan, index=merged.index)  # créer série vide

  rel_error[both_zero] = 0
  rel_error[normal] = ((ref[normal] - comp[normal])/ ref[normal]).abs()*100

  max = rel_error.max()
  min = rel_error.min()
  mean = rel_error.mean(skipna=True)
  q = rel_error.quantile(0.8)

  idx_max = rel_error.idxmax()
  country = merged.loc[idx_max, "Country Name"]
  year = merged.loc[idx_max, "Year"]
  val_ref = merged.loc[idx_max, "Value_ref"]
  val_comp = merged.loc[idx_max, "Value_comp"]

  return max, min, mean, q, country, year, val_ref, val_comp

In [16]:
ind = "Agricultural land (% of land area)"
missing_per_ind(ind)

(np.int64(2124), np.float64(15.27))

In [17]:
missing_per()

(np.int64(689231), np.float64(56.31))

In [18]:
ESG_df['Country Name'].unique()

array(['Cameroon', 'Qatar', 'Paraguay', 'Portugal',
       "Korea, Dem. People's Rep.", 'Poland', 'Papua New Guinea', 'Palau',
       'Philippines', 'Congo, Dem. Rep.', 'Romania', 'Peru', 'Pakistan',
       'Oman', 'New Zealand', 'Nauru', 'Nepal', 'Norway', 'Honduras',
       'Croatia', 'Haiti', 'Panama', 'Hungary', 'Russian Federation',
       'Saudi Arabia', 'Togo', 'Chad', 'Syrian Arab Republic',
       'Seychelles', 'Eswatini', 'Sweden', 'Slovenia', 'Slovak Republic',
       'Suriname', 'Rwanda', 'Sao Tome and Principe', 'Serbia', 'Somalia',
       'San Marino', 'El Salvador', 'Sierra Leone', 'Solomon Islands',
       'Singapore', 'Senegal', 'Sudan', 'South Sudan', 'Indonesia',
       'India', 'Ireland', 'Monaco', 'Moldova', 'Madagascar', 'Maldives',
       'Mexico', 'Marshall Islands', 'North Macedonia', 'Mali', 'Malta',
       'Morocco', 'Myanmar', 'Mongolia', 'Mozambique', 'Mauritania',
       'Mauritius', 'Malawi', 'Malaysia', 'Namibia', 'Niger', 'Nigeria',
       'Montenegro',

# Integracion de datos

## Sources utilisées :

Ancien Dataset principal : World Bank Group. (s. d.). Environment, Social and Governance (ESG) Data – bulk download (ESG_CSV.zip). Retrieved October 22, 2025, from https://databankfiles.worldbank.org/public/ddpext_download/ESG_CSV.zip

Nouveau : World Bank Group. (s. d.). Download data – Sovereign ESG Data Portal [Dataset]. Retrieved 29 October 2025, from https://esgdata.worldbank.org/data/download?lang=en

- Access to clean fuels and technologies for cooking (% of population)

Data sources: World Health Organization - Global Health Observatory (2024) – processed by Our World in Data https://ourworldindata.org/grapher/access-to-clean-fuels-and-technologies-for-cooking?time=latest

- Access to electricity (% of population)

Les autres datasets ne présentent pas de nouvelles données

- Adjusted savings: natural resources depletion (% of GNI)

Les autres datasets ne présentent pas de nouvelles données

- Adjusted savings: net forest depletion (% of GNI)

Les autres datasets ne présentent pas de nouvelles données

- Agricultural land (% of land area)

Food and Agriculture Organization of the United Nations (FAO). (2025). FAOSTAT – Land Use. Retrieved October 3, 2025, from https://www.fao.org/faostat/en/#data/RL

- Agriculture, forestry, and fishing, value added (% of GDP)

Les autres datasets ne présentent pas de nouvelles données

- Annual freshwater withdrawals, total (% of internal resources)

Choisi finalement de ne pas inclure les données (seulement 238 nouvelles données + problème avec des valeurs infinies lors du calcul)

Food and Agriculture Organization of the United Nations (FAO). (n.d.). AQUASTAT – FAO’s Global Information System on Water and Agriculture. Retrieved October 3, 2025, from https://data.apps.fao.org/aquastat/?lang=en

Note: Data have been processed and analyzed by the author.

- Annualized average growth rate in per capita real survey mean consumption or income, total population (%)

United Nations, Department of Economic and Social Affairs, Statistics Division. (n.d.). Annualized average growth rate in per capita real survey mean consumption or income, total population (%) [Data set]. UNdata. Retrieved October 3, 2025, from https://data.un.org/Data.aspx?d=WDI&f=Indicator_Code%3ASI.SPR.PCAP.ZG

Pas de valeur annuelle car calculé par sondage entre deux années (explique la quantité de valeur manquante), peut servir au clustering

- Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)

Les données concernent bien les communicable diseases and maternal, prenatal and nutrition conditions (filtré lors de la récupération des données)

On remarque une différence moyenne relativement élevée quoi qu'acceptable (14.51%). On a un 80e centile à 21% avec une valeur extrême max à 150%. Il faut prendre en compte le fait que l'indicateur est particulièrement vide pour le dataset d'origine, ce qui peut expliquer cette différence. De plus le nouveau dataset permet de rendre l'indicateur utilisable en ajoutant 4000 valeur. Il est donc choisi d'utiliser ce dataset après un recalibrage et en eliminant les extremums (ce qui sera de toute façon fait par la suite)

World Health Organization. (n.d.). Mortality Database: Communicable, maternal, perinatal and nutritional conditions. Retrieved October 5, 2025, from https://platform.who.int/mortality/themes/theme-details/MDB/communicable-maternal-perinatal-and-nutritional-conditions

- Children in employment, total (% of children ages 7-14)

Les autres datasets ne présentent pas de nouvelles données ou présentent des données sur une période (2002-2011)

- Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita)

On remarque une différence moyenne un peu élevée quoi qu'acceptable (10.29%). On a un 80e centile à 11.59% avec une valeur extrême max à 6200% (clairement une erreur). Le nouveau dataset permet d'ajouter 6000 valeurs et complète l'indicateur le plus important. Il est donc choisi d'utiliser ce dataset après un recalibrage et en eliminant les extremums (ce qui sera de toute façon fait par la suite)

Data sources: Global Carbon Budget (2024) Population based on various sources (2024) – with major processing by Our World in Data https://ourworldindata.org/grapher/co-emissions-per-capita


https://globalcarbonbudgetdata.org/latest-data.html
Global Carbon Budget v2024 [xlsx]

- Coastal protection

Les autres datasets ne présentent pas de nouvelles données

- Control of Corruption: Estimate

Les autres datasets ne présentent pas de nouvelles données

- Cooling Degree Days

Le nouveau dataset présente une trop grosse différence avec celui d'origine (80%) non utilisable + indicateur assez complet (80%)

- Economic and Social Rights Performance Score

Pas réussi à collecter de nouvelles données

- Electricity production from coal sources (% of total)

On remarque une différence moyenne assez faible (7.02%). On a un 80e centile très bas à 2.53% avec une valeur extrême max à 1724.62% (probablement une erreur). Il est donc choisi d'utiliser ce dataset après un recalibrage et en eliminant les extremums (ce qui sera de toute façon fait par la suite)

Data sources: Ember (2025)Energy Institute - Statistical Review of World Energy (2025) – with major processing by Our World in Data https://ourworldindata.org/grapher/share-electricity-coal?overlay=download-data

- Energy imports, net (% of energy use)

Les autres datasets ne présentent pas de nouvelles données

- Energy intensity level of primary energy (MJ/$2017 PPP GDP)

Les autres datasets ne présentent pas de nouvelles données

- Energy use (kg of oil equivalent per capita)

Les autres datasets ne présentent pas de nouvelles données

- Fertility rate, total (births per woman)

L'indicateur est déjà complet

- Food production index (2014-2016 = 100)

Les autres datasets ne présentent pas de nouvelles données

- Forest area (% of land area)

Data sources: Department for Environment, Food & Rural Affairs (2013)Food and Agriculture Organization of the United Nations Regional Office for Asia and the Pacific (2009)Forest Research (2002)Mather A.S., Fairbairn J., and Needle C.J. (1999)Osamu Saito (2009)Yi-Ying Chen et al. (2019)A.S. Mather (2008)Kleinn, C., Corrales, L., & Morales, D. (2002)Soo Bae J., Won Joo, R. and Kim, Y.S. (2012)United States Department of Agriculture, Forest Service (2014)He, F., Yang, F, & Wang, Y. (2024)Scottish Government (2019)Food and Agriculture Organization of the United Nations (2024) – with major processing by Our World in Data https://ourworldindata.org/grapher/forest-area-as-share-of-land-area

- Fossil fuel energy consumption (% of total)

Data sources: Energy Institute - Statistical Review of World Energy (2025) – with major processing by Our World in Data https://ourworldindata.org/fossil-fuels

On remarque une différence moyenne relativement faible (10.39%). On a un 80e centile à 13.06% avec une valeur extrême max à 200.16%. Ce dataset ne présente pas exactement le même indicateur (méthode de substitution) mais permet de bien compléter l'ind. A voir si on utilise ou impute

- GDP growth (annual %)

Choisi de ne pas compléter celui là, grosse différence moyenne (91.23%) et assez gros 80e centile (12%), de plus ajoute seulement 100 valeurs, pas très intéressant, va juste ajuster du bruit. La différence viendrait du fait que le FMI utilise la monnaie nationale contrairement à world bank. Peut être voir imputation.

- GHG net emissions/removals by LUCF (Mt of CO2 equivalent)

Données trop différentes

- Gini index

World Population Review. (2025). Gini Coefficient by Country. Retrieved October 6, 2025, from https://worldpopulationreview.com/country-rankings/gini-coefficient-by-country

Organisation for Economic Co-operation and Development (OECD). (2025). Income Inequality Indicators. Retrieved October 6, 2025, from https://www.oecd.org/en/data/indicators/income-inequality.html

World Bank. (n.d.). Gini Index (SI.POV.GINI). In World Development Indicators. Retrieved October 6, 2025, from https://data.un.org/Data.aspx?d=WDI&f=Indicator_Code%3ASI.POV.GINI

- Government Effectiveness: Estimate

Data sources: V-Dem (2025)Our World in Data – processed by Our World in Data

- Government expenditure on education, total (% of government expenditure)

Les autres datasets ne présentent pas de nouvelles données

- Heat Index 35

Les autres datasets ne présentent pas de nouvelles données

- Heating Degree Days

Les autres datasets ne présentent pas de nouvelles données

- Hospital beds (per 1,000 people)

World Health Organization. (n.d.). Hospital beds (per 10 000 population). Retrieved October 12, 2025, from https://www.who.int/data/gho/data/indicators/indicator-details/GHO/hospital-beds-(per-10-000-population)

- Income share held by lowest 20%

World Bank. (n.d.). Income share held by lowest 20% (SI.DST.FRST.20). In World Development Indicators. Retrieved October 6, 2025, from https://data.un.org/Data.aspx?d=WDI&f=Indicator_Code%3ASI.DST.FRST.20

- Individuals using the Internet (% of population)

Data sources: International Telecommunication Union (ITU), via World Bank (2025) – with minor processing by Our World in Data https://ourworldindata.org/grapher/share-of-individuals-using-the-internet?tab=line&country=~FRA&overlay=download-data

- Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)

International Labour Organization. (n.d.). Labour force participation rate (ILOSTAT snapshot). Retrieved October 12, 2025, from https://ilostat.ilo.org/data/snapshots/labour-force-participation-rate/

- Land Surface Temperature

Les données du nouveau dataset sont trop différentes (30%)

- Level of water stress: freshwater withdrawal as a proportion of available freshwater resources

Food and Agriculture Organization of the United Nations (FAO). (n.d.). AQUASTAT – FAO’s Global Information System on Water and Agriculture. Retrieved October 12, 2025, from https://data.apps.fao.org/aquastat/?lang=en

- Life expectancy at birth, total (years)

Indicateur complet

- Literacy rate, adult total (% of people ages 15 and above)

World Population Review. (2025). Literacy Rate by Country. Retrieved October 13, 2025, from https://worldpopulationreview.com/country-rankings/literacy-rate-by-country

- Mammal species, threatened

World Bank. (n.d.). Number of threatened mammal species (EN.MAM.THRD.NO). Retrieved October 13, 2025, from https://data.worldbank.org/indicator/EN.MAM.THRD.NO?most_recent_value_desc=false

Un peu particulier, une seule valeur pour une années (une constante), soit la mapper pour toute les années soit l'utiliser en clustering ou imputer le reste ?

- Methane emissions (metric tons of CO2 equivalent per capita)

Trop gros écart avec les nouvelles données (60%)

- Mortality rate, under-5 (per 1,000 live births)

Data sources: United Nations Inter-agency Group for Child Mortality Estimation (2025)Gapminder (2015)Gapminder based on UN IGME & UN WPP (2020)Population based on various sources (2024) – processed by Our World in Data

- Net migration

Indicateur complet

- Nitrous oxide emissions (metric tons of CO2 equivalent per capita)

Trop gros écart avec les nouvelles données (30%)

- Patent applications, residents

Les autres datasets ne présentent pas de nouvelles données

- People using safely managed drinking water services (% of population)

Les autres datasets ne présentent pas de nouvelles données

- People using safely managed sanitation services (% of population)

Les autres datasets ne présentent pas de nouvelles données

- PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)

Les autres datasets ne présentent pas de nouvelles données

- Political Stability and Absence of Violence/Terrorism: Estimate

Les autres datasets ne présentent pas de nouvelles données

- Population ages 65 and above (% of total population)

Indicateur complet

- Population density (people per sq. km of land area)

Data sources: HYDE (2023)Gapminder - Population v7 (2022)UN, World Population Prospects (2024)Gapminder - Systema Globalis (2022)Food and Agriculture Organization of the United Nations (2024) – with major processing by Our World in Data https://ourworldindata.org/grapher/population-density?tab=line&time=1979..latest&country=~FRA&overlay=download-data

- Poverty headcount ratio at national poverty lines (% of population)

DataHub. (n.d.). Poverty headcount ratio at national poverty lines (% of population) (SI.POV.NAHC). Retrieved October 13, 2025, from https://datahub.io/core/world-development-indicators/indicators/si.pov.nahc

- Prevalence of overweight (% of adults)

Les autres datasets ne présentent pas de nouvelles données

- Prevalence of undernourishment (% of population)

Food and Agriculture Organization of the United Nations (FAO). (n.d.). FAO Data Explorer: Hunger (SDG 2.1.1) – Prevalence of undernourishment (A.SN_ITK_DEFC). Retrieved October 13, 2025, from https://dataexplorer.fao.org/vis?fs[0]=Sustainable%20Development%20Goals%20%28SDGs%29%2C1%7CGoal%202%20Zero%20Hunger%23SDG_G2%23%7C2.1.1%20Hunger%23SDG_G2_211%23&pg=0&fc=Sustainable%20Development%20Goals%20%28SDGs%29&bp=true&snb=1&vw=ov&df[ds]=ds-release&df[id]=DF_SDG_2_1_1&df[ag]=FAO&df[vs]=1.0&dq=A.SN_ITK_DEFC..........&pd=2020%2C2023&to[TIME_PERIOD]=false

- Proportion of bodies of water with good ambient water quality

UN-Water SDG 6 Data. (n.d.). Indicator 6.3.2: Proportion of bodies of water with good ambient water quality. Retrieved October 13, 2025, from https://sdg6data.org/indicator/6.3.2

On remarque une différence moyenne relativement faible (11.11%). On a un 80e centile à 20.0% avec une valeur extrême max à 33%. On va surement conserver les données, les indicateurs semblent cohérent et la diff peut s'expliquer par le manque de données. Pas utilisable en régréssion mais à voir en clustering

- Proportion of seats held by women in national parliaments (%)

United Nations Statistics Division. (n.d.). UN SDG Data Portal / UNStats database. Retrieved October 13, 2025, from https://unstats.un.org/sdgs/dataportal/database

- Ratio of female to male labor force participation rate (%) (modeled ILO estimate)

International Labour Organization. (n.d.). ILOSTAT database. Retrieved October 13, 2025, from https://ilostat.ilo.org/data/
Erreur moyenne faible 6.3% et 80e centile de 8.52%, quelques outliers à supprimer

- Regulatory Quality: Estimate

Les autres datasets ne présentent pas de nouvelles données

- Renewable electricity output (% of total electricity output)

Données trop différentes (20%)

- Renewable energy consumption (% of total final energy consumption)

United Nations Economic Commission for Europe (UNECE). (n.d.). UNECE SDG Indicators: Indicator 23. Retrieved October 13, 2025, from https://w3.unece.org/SDG/en/Indicator?id=23

and

Eurostat. (n.d.). Renewable energy statistics. Retrieved October 13, 2025, from https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Renewable_energy_statistics

- Research and development expenditure (% of GDP)

OECD. (n.d.). Gross domestic spending on R&D. Retrieved October 17, 2025, from https://www.oecd.org/en/data/indicators/gross-domestic-spending-on-r-d.html

- Rule of Law: Estimate

Les autres datasets ne présentent pas de nouvelles données

- School enrollment, primary (% gross)

UNESCO Institute for Statistics (UIS). (n.d.). Gross Enrolment Ratio (GER) — Education Data Browser. Retrieved October 13, 2025, from https://databrowser.uis.unesco.org/browser/EDUCATION/UIS-EducationOPRI/enrol-att/enrolment/IG-GER?highlightGroupId=IG-GER&highlightId=GER.1

- School enrollment, primary and secondary (gross), gender parity index (GPI)

Data sources: UNESCO Institute for Statistics (2025) – with minor processing by Our World in Data https://ourworldindata.org/grapher/net-enrollment-rate-primary-gender-parity-index-gpi?tab=line&country=~ARG&overlay=download-data

- Scientific and technical journal articles

Les autres datasets ne présentent pas de nouvelles données

- Standardised Precipitation-Evapotranspiration Index

Indicateur quasiment complet (6%)

- Strength of legal rights index (0=weak to 12=strong)

Les autres datasets ne présentent pas de nouvelles données

- Terrestrial and marine protected areas (% of total territorial area)

Les autres datasets ne présentent pas de nouvelles données

- Tree Cover Loss (hectares)

Data sources: Global Forest Watch (2025) – with major processing by Our World in Data https://ourworldindata.org/grapher/tree-cover-loss?tab=line&country=~AFG&overlay=download-data

- Unemployment, total (% of total labor force) (modeled ILO estimate)

Les autres datasets ne présentent pas de nouvelles données

- Unmet need for contraception (% of married women ages 15-49)

United Nations, Department of Economic and Social Affairs. (n.d.). Population Division – Data portal [Page]. Retrieved October 13, 2025, from https://www.un.org/development/desa/pd/node/3288

DataHub. (n.d.). Unmet need for contraception (% of married women ages 15-49) (SP.UWT.TFRT). Retrieved October 13, 2025, from https://datahub.io/core/world-development-indicators/indicators/sp.uwt.tfrt

- Voice and Accountability: Estimate

Les autres datasets ne présentent pas de nouvelles données

## Récupération des données

### Access to clean fuels and technologies for cooking (% of population)

In [19]:
# Access to clean fuels and technologies for cooking (% of population)
url = "https://ourworldindata.org/grapher/access-to-clean-fuels-and-technologies-for-cooking.csv?v=1&csvType=full&useColumnShortNames=true"
access_to_clean_fuels_and_technologies_for_cooking = pd.read_csv(url, storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Limpezia del dataset
access_to_clean_fuels_and_technologies_for_cooking.drop(columns=["Code"], inplace=True)
access_to_clean_fuels_and_technologies_for_cooking.rename(columns={"Entity": "Country Name",
"proportion_of_population_with_primary_reliance_on_clean_fuels_and_technologies_for_cooking__pct__residence_area_type_total":"Access to clean fuels and technologies for cooking (% of population)"}, inplace=True)
access_to_clean_fuels_and_technologies_for_cooking['Country Name'] = access_to_clean_fuels_and_technologies_for_cooking['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Access to clean fuels and technologies for cooking (% of population)"] = access_to_clean_fuels_and_technologies_for_cooking

# Verificar la forma
access_to_clean_fuels_and_technologies_for_cooking.head()

,Country Name,Year,Access to clean fuels and technologies for cooking (% of population)
0,Afghanistan,2000,6.20
1,Afghanistan,2001,7.10
2,Afghanistan,2002,8.30
3,Afghanistan,2003,9.35
4,Afghanistan,2004,10.80


### Agricultural land (% of land area)

In [20]:
# Agricultural land (% of land area)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Agricultural%20land%20(%25%20of%20land%20area)/FAOSTAT_LandUse.csv"

agricultural_land = pd.read_csv(url,encoding='utf-8')

# Supprimer colonnes et lignes inutiles
agricultural_land = agricultural_land[(agricultural_land["Item"] == "Agricultural land") & (agricultural_land["Element"] == "Share in Land area")]
agricultural_land.drop(columns=["Area Code","Area Code (M49)","Item","Item Code", "Element Code", "Element", "Unit"], inplace=True)

agricultural_land = agricultural_land.rename(columns={"Area": "Country Name"})

# Changer le format pour correspondre aux autres dataframe
agricultural_land = agricultural_land.melt(
    id_vars=["Country Name"],
    var_name="Year",
    value_name="Agricultural land (% of land area)"
)

agricultural_land["Year"] = agricultural_land["Year"].str.replace("Y","").astype(int)

# Dans ce dataset China représente la Chine avec Hong Kong et Taiwan ce qui n'est pas le cas de world bank, on veut donc conserver uniquement china,mainland
agricultural_land = agricultural_land[agricultural_land['Country Name']!="China"]
agricultural_land['Country Name'] = agricultural_land['Country Name'].replace(country_name_map)

# On supprimer les NaN car avec Ethiopia et Ethiopia PDR on a des doublons
agricultural_land = agricultural_land.dropna()

# Add the dataset to the list
new_datas["Agricultural land (% of land area)"] = agricultural_land

# Verificar la forma
agricultural_land.head()

,Country Name,Year,Agricultural land (% of land area)
0,Afghanistan,1961,57.88
1,Albania,1961,44.96
2,Algeria,1961,19.09
3,American Samoa,1961,9.65
4,Andorra,1961,55.32


### Annual freshwater withdrawals, total (% of internal resources)

In [21]:
'''# Annual freshwater withdrawals, total (% of internal resources)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Annual%20freshwater%20withdrawals,%20total%20(%25%20of%20internal%20resources)/FAO_AQUASTAT.csv"
annual_freshwater_withdrawals_total = pd.read_csv(url,encoding='utf-8')

# Calculer l'indicateur
annual_freshwater_withdrawals_total["Annual freshwater withdrawals, total (% of internal resources)"] = annual_freshwater_withdrawals_total["Total freshwater withdrawal [10^9 m3/year]"]/(annual_freshwater_withdrawals_total["Total renewable water resources [10^9 m3/year]"]-annual_freshwater_withdrawals_total["Water resources: total external renewable [10^9 m3/year]"])*100

# Nettoyage
annual_freshwater_withdrawals_total.drop(columns=["Total freshwater withdrawal [10^9 m3/year]","Total renewable water resources [10^9 m3/year]","Water resources: total external renewable [10^9 m3/year]"], inplace=True)
annual_freshwater_withdrawals_total.rename(columns={"AREA":"Country Name","timePointYears":"Year"}, inplace=True)

annual_freshwater_withdrawals_total['Country Name'] = annual_freshwater_withdrawals_total['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Annual freshwater withdrawals, total (% of internal resources)"] = annual_freshwater_withdrawals_total

# Verificar la forma
annual_freshwater_withdrawals_total.head()'''

'# Annual freshwater withdrawals, total (% of internal resources)\nurl = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Annual%20freshwater%20withdrawals,%20total%20(%25%20of%20internal%20resources)/FAO_AQUASTAT.csv"\nannual_freshwater_withdrawals_total = pd.read_csv(url,encoding=\'utf-8\')\n\n# Calculer l\'indicateur\nannual_freshwater_withdrawals_total["Annual freshwater withdrawals, total (% of internal resources)"] = annual_freshwater_withdrawals_total["Total freshwater withdrawal [10^9 m3/year]"]/(annual_freshwater_withdrawals_total["Total renewable water resources [10^9 m3/year]"]-annual_freshwater_withdrawals_total["Water resources: total external renewable [10^9 m3/year]"])*100\n\n# Nettoyage\nannual_freshwater_withdrawals_total.drop(columns=["Total freshwater withdrawal [10^9 m3/year]","Total renewable water resources [10^9 m3/year]","Water resources: total external renewable [10^9 m3/year]"], inplace=T

### Annualized average growth rate in per capita real survey mean consumption or income, total population (%)

In [22]:
# Annualized average growth rate in per capita real survey mean consumption or income, total population (%)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Annualized%20average%20growth%20rate%20in%20per%20capita%20real%20survey%20mean%20consumption%20or%20income,%20total%20population%20(%25)/UNdata_average_growth_rate.csv"
annualized_average_growth_rate = pd.read_csv(url,encoding='utf-8')

# Nettoyage
annualized_average_growth_rate.drop(columns=["Value Footnotes"], inplace=True)
annualized_average_growth_rate.rename(columns={"Country or Area":"Country Name","Value":"Annualized average growth rate in per capita real survey mean consumption or income, total population (%)"}, inplace=True)

annualized_average_growth_rate['Country Name'] = annualized_average_growth_rate['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Annualized average growth rate in per capita real survey mean consumption or income, total population (%)"] = annualized_average_growth_rate

# Verificar la forma
annualized_average_growth_rate.head()

,Country Name,Year,"Annualized average growth rate in per capita real survey mean consumption or income, total population (%)"
0,Albania,2020,2.36
1,Argentina,2022,-4.71
2,Armenia,2018,2.40
3,Australia,2018,-0.26
4,Austria,2021,0.35


### Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)

In [23]:
# Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Cause%20of%20death,%20by%20communicable%20diseases%20and%20maternal,%20prenatal%20and%20nutrition%20conditions%20(%25%20of%20total)/WHOMortalityDatabase.csv"
cause_of_death = pd.read_csv(url,encoding='utf-8')

# Limpezia del dataset
cause_of_death = cause_of_death[(cause_of_death["Sex"] == "All") & (cause_of_death["Age Group"] == "[All]")]
cause_of_death.drop(columns=cause_of_death.columns.drop(["Country Name","Year","Percentage of cause-specific deaths out of total deaths"]), inplace=True)
cause_of_death.rename(columns={"Percentage of cause-specific deaths out of total deaths":
                               "Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)"}, inplace=True)

cause_of_death['Country Name'] = cause_of_death['Country Name'].replace(country_name_map)
# Add the dataset to the list
new_datas["Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)"] = cause_of_death

# Verificar la forma
cause_of_death.head()

,Country Name,Year,"Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)"
0,Albania,1987,17.280388
3,Albania,1988,15.405546
6,Albania,1989,17.876603
9,Albania,1992,15.703678
12,Albania,1993,16.888034


### Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita)

In [24]:
'''# Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita)
url = "https://ourworldindata.org/grapher/co-emissions-per-capita.csv?v=1&csvType=full&useColumnShortNames=false"
co2_emissions = pd.read_csv(url, storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Limpezia del dataset
co2_emissions.drop(columns=["Code"], inplace=True)
co2_emissions.rename(columns={"Entity": "Country Name","Annual CO₂ emissions (per capita)" : "Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita)"}, inplace=True)
co2_emissions['Country Name'] = co2_emissions['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita)"] = co2_emissions

# Verificar la forma
co2_emissions.head()'''

'# Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita)\nurl = "https://ourworldindata.org/grapher/co-emissions-per-capita.csv?v=1&csvType=full&useColumnShortNames=false"\nco2_emissions = pd.read_csv(url, storage_options = {\'User-Agent\': \'Our World In Data data fetch/1.0\'})\n\n# Limpezia del dataset\nco2_emissions.drop(columns=["Code"], inplace=True)\nco2_emissions.rename(columns={"Entity": "Country Name","Annual CO₂ emissions (per capita)" : "Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita)"}, inplace=True)\nco2_emissions[\'Country Name\'] = co2_emissions[\'Country Name\'].replace(country_name_map)\n\n# Add the dataset to the list\nnew_datas["Carbon dioxide (CO2) emissions excluding LULUCF per capita (t CO2e/capita)"] = co2_emissions\n\n# Verificar la forma\nco2_emissions.head()'

### Electricity production from coal sources (% of total)

In [25]:
# Electricity production from coal sources (% of total)
url = "https://ourworldindata.org/grapher/share-electricity-coal.csv?v=1&csvType=full&useColumnShortNames=false"
electricity_production_from_coal_sources = pd.read_csv(url, storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Limpezia del dataset
electricity_production_from_coal_sources.drop(columns=["Code"], inplace=True)
electricity_production_from_coal_sources.rename(columns={"Entity": "Country Name","Coal - % electricity" : "Electricity production from coal sources (% of total)"}, inplace=True)
electricity_production_from_coal_sources['Country Name'] = electricity_production_from_coal_sources['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Electricity production from coal sources (% of total)"] = electricity_production_from_coal_sources

# Verificar la forma
electricity_production_from_coal_sources.head()

,Country Name,Year,Electricity production from coal sources (% of total)
0,ASEAN (Ember),2000,20.081350
1,ASEAN (Ember),2001,21.306658
2,ASEAN (Ember),2002,21.567904
3,ASEAN (Ember),2003,22.261261
4,ASEAN (Ember),2004,23.161350


### Forest area (% of land area)

In [26]:
# Forest area (% of land area)
url = "https://ourworldindata.org/grapher/forest-area-as-share-of-land-area.csv?v=1&csvType=full&useColumnShortNames=false"
forest_area = pd.read_csv(url, storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Limpezia del dataset
forest_area.drop(columns=["Code","1039812-annotations"], inplace=True)
forest_area.rename(columns={"Entity": "Country Name","Share of land covered by forest":"Forest area (% of land area)"}, inplace=True)
forest_area['Country Name'] = forest_area['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Forest area (% of land area)"] = forest_area

# Verificar la forma
forest_area.head()

,Country Name,Year,Forest area (% of land area)
0,Afghanistan,1990,1.850994
1,Afghanistan,1991,1.850994
2,Afghanistan,1992,1.850994
3,Afghanistan,1993,1.850994
4,Afghanistan,1994,1.850994


### Fossil fuel energy consumption (% of total)

In [27]:
# Fossil fuel energy consumption (% of total)
url = "https://ourworldindata.org/grapher/fossil-fuels-share-energy.csv?v=1&csvType=full&useColumnShortNames=false"
fossil_fuel_energy_consumption = pd.read_csv(url, storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Limpezia del dataset
fossil_fuel_energy_consumption.drop(columns=["Code"], inplace=True)
fossil_fuel_energy_consumption.rename(columns={"Entity": "Country Name","Fossil fuels (% equivalent primary energy)":"Fossil fuel energy consumption (% of total)"}, inplace=True)
fossil_fuel_energy_consumption['Country Name'] = fossil_fuel_energy_consumption['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Fossil fuel energy consumption (% of total)"] = fossil_fuel_energy_consumption

# Verificar la forma
fossil_fuel_energy_consumption.head()

,Country Name,Year,Fossil fuel energy consumption (% of total)
0,Africa,1965,94.580734
1,Africa,1966,94.227600
2,Africa,1967,94.036240
3,Africa,1968,93.394530
4,Africa,1969,92.494280


### Gini index

In [28]:
# Gini index
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Gini%20Index/OECD_2012.csv"
gini_index = pd.read_csv(url)

# Limpezia del dataset
cols_to_drop = gini_index.columns.drop(["Zone de référence","TIME_PERIOD","OBS_VALUE"])
gini_index.drop(columns=cols_to_drop, inplace=True)
gini_index.rename(columns={"Zone de référence": "Country Name","TIME_PERIOD":"Year","OBS_VALUE":"Gini index"}, inplace=True)

gini_index['Country Name'] = gini_index['Country Name'].replace(country_name_map)
gini_index['Gini index'] = gini_index['Gini index']*100

# 2e Dataset
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Gini%20Index/UNdata_Export_20251011_155542293.csv"
gini_index_2 = pd.read_csv(url)

gini_index_2.drop(columns=["Value Footnotes"], inplace=True)
gini_index_2.rename(columns={"Country or Area": "Country Name","Value":"Gini index"}, inplace=True)
gini_index_2['Country Name'] = gini_index_2['Country Name'].replace(country_name_map)
gini_index_2['Country Name'] = gini_index_2['Country Name'].replace({"Micronesia" : "Micronesia, Fed. Sts."})

gini_index = pd.merge(gini_index,
                 gini_index_2,
                 on=["Country Name","Year"],
                 how="outer",
                 suffixes=("", "_new"))

gini_index["Gini index"] = gini_index["Gini index"].fillna(
    gini_index["Gini index_new"]
)

# Supprimer la colonne temporaire
gini_index.drop(columns=["Gini index_new"], inplace=True)

#3e Dataset
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Gini%20Index/gini-coefficient-by-country-2025.csv"
gini_index_3 = pd.read_csv(url)

gini_index_3.drop(columns=["flagCode"], inplace=True)
gini_index_3.rename(columns={"country": "Country Name","GiniCoefficientByCountryMostRecentUpdateYear":"Year","GiniCoefficientByCountry":"Gini index"}, inplace=True)
gini_index_3['Country Name'] = gini_index_3['Country Name'].replace(country_name_map)
gini_index_3['Country Name'] = gini_index_3['Country Name'].replace({"Micronesia" : "Micronesia, Fed. Sts."})

gini_index = pd.merge(gini_index,
                 gini_index_3,
                 on=["Country Name","Year"],
                 how="outer",
                 suffixes=("", "_new"))

gini_index["Gini index"] = gini_index["Gini index"].fillna(
    gini_index["Gini index_new"]
)

# Supprimer la colonne temporaire
gini_index.drop(columns=["Gini index_new"], inplace=True)

# Add the dataset to the list
new_datas["Gini index"] = gini_index

# Verificar la forma
gini_index.head()

,Country Name,Year,Gini index
0,Albania,1996,27.0
1,Albania,2002,31.7
2,Albania,2005,30.6
3,Albania,2008,30.0
4,Albania,2012,29.0


### Government Effectiveness: Estimate

In [29]:
# Government Effectiveness: Estimate
url = "https://ourworldindata.org/grapher/government-effectiveness-vs-electoral-democracy-index.csv?v=1&csvType=full&useColumnShortNames=true"
government_effectiveness = pd.read_csv(url, storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Limpezia del dataset
government_effectiveness.drop(columns=["Code","electdem_vdem__estimate_best","owid_region"], inplace=True)
government_effectiveness.rename(columns={"Entity": "Country Name","goveffective_vdem_wbgi":"Government Effectiveness: Estimate"}, inplace=True)
government_effectiveness['Country Name'] = government_effectiveness['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Government Effectiveness: Estimate"] = government_effectiveness

# Verificar la forma
government_effectiveness.head()

,Country Name,Year,Government Effectiveness: Estimate
0,Afghanistan,1789,NaN
1,Afghanistan,1790,NaN
2,Afghanistan,1791,NaN
3,Afghanistan,1792,NaN
4,Afghanistan,1793,NaN


###Hospital beds (per 1,000 people)

In [30]:
# Hospital beds (per 1,000 people)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Hospital%20beds%20(per%201,000%20people)/WHO_hospital_bed.csv"
hospital_beds = pd.read_csv(url)

# Limpezia del dataset
cols_to_drop = hospital_beds.columns.drop(["Location","Period","FactValueNumeric"])
hospital_beds.drop(columns=cols_to_drop, inplace=True)
hospital_beds.rename(columns={"Location": "Country Name","Period":"Year","FactValueNumeric":"Hospital beds (per 1,000 people)"}, inplace=True)

hospital_beds['Country Name'] = hospital_beds['Country Name'].replace(country_name_map)

hospital_beds["Hospital beds (per 1,000 people)"] = hospital_beds["Hospital beds (per 1,000 people)"]/10

# Add the dataset to the list
new_datas["Hospital beds (per 1,000 people)"] = hospital_beds

# Verificar la forma
hospital_beds.head()

,Country Name,Year,"Hospital beds (per 1,000 people)"
0,Bangladesh,2022,1.013
1,Eritrea,2022,1.040
2,Oman,2022,1.047
3,Belize,2022,1.068
4,El Salvador,2022,1.203


###Income share held by lowest 20%

In [31]:
#Income share held by lowest 20%
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Income%20share%20held%20by%20lowest%2020%25/UNdata_Export_20250915_013922709.csv"
income_share_held_by_lowest_20 = pd.read_csv(url)

# Limpezia del dataset
income_share_held_by_lowest_20.drop(columns=["Value Footnotes"], inplace=True)
income_share_held_by_lowest_20.rename(columns={"Country or Area": "Country Name","Value":"Income share held by lowest 20%"}, inplace=True)

income_share_held_by_lowest_20['Country Name'] = income_share_held_by_lowest_20['Country Name'].replace(country_name_map)
income_share_held_by_lowest_20['Country Name'] = income_share_held_by_lowest_20['Country Name'].replace({"Micronesia" : "Micronesia, Fed. Sts."})

# Add the dataset to the list
new_datas["Income share held by lowest 20%"] = income_share_held_by_lowest_20

# Verificar la forma
income_share_held_by_lowest_20.head()

,Country Name,Year,Income share held by lowest 20%
0,Albania,2020,8.4
1,Albania,2019,8.5
2,Albania,2018,7.9
3,Albania,2017,7.6
4,Albania,2016,7.3


###Individuals using the Internet (% of population)

In [32]:
# Individuals using the Internet (% of population)
url = "https://ourworldindata.org/grapher/share-of-individuals-using-the-internet.csv?v=1&csvType=full&useColumnShortNames=false"
individuals_using_the_internet = pd.read_csv(url, storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Limpezia del dataset
individuals_using_the_internet.drop(columns=["Code"], inplace=True)
individuals_using_the_internet.rename(columns={"Entity": "Country Name"}, inplace=True)

individuals_using_the_internet['Country Name'] = individuals_using_the_internet['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Individuals using the Internet (% of population)"] = individuals_using_the_internet

# Verificar la forma
individuals_using_the_internet.head()

,Country Name,Year,Individuals using the Internet (% of population)
0,Afghanistan,1990,0.0
1,Afghanistan,1991,0.0
2,Afghanistan,1992,0.0
3,Afghanistan,1993,0.0
4,Afghanistan,1994,0.0


###Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)

In [33]:
# Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Labor%20force%20participation%20rate,%20total%20(%25%20of%20total%20population%20ages%2015-64)%20(modeled%20ILO%20estimate)/EAP_DWAP_SEX_AGE_RT_A-20250915T2011.csv"
labor_force_participation_rate = pd.read_csv(url)

# Limpezia del dataset
labor_force_participation_rate = labor_force_participation_rate[(labor_force_participation_rate['sex.label'] == 'Total') & (labor_force_participation_rate['classif1.label'] == 'Age (Youth, adults): 15-64')]

cols_to_drop = labor_force_participation_rate.columns.drop(["ref_area.label","time","obs_value"])
labor_force_participation_rate.drop(columns=cols_to_drop, inplace=True)
labor_force_participation_rate.rename(columns={"ref_area.label": "Country Name","time":"Year","obs_value":"Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)"}, inplace=True)

labor_force_participation_rate['Country Name'] = labor_force_participation_rate['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)"] = labor_force_participation_rate

# Verificar la forma
labor_force_participation_rate.head()

,Country Name,Year,"Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)"
79,Aruba,2007,73.760
166,Aruba,1991,67.170
268,Afghanistan,2021,50.851
352,Afghanistan,2020,42.786
436,Afghanistan,2017,48.744


###Level of water stress: freshwater withdrawal as a proportion of available freshwater resources

In [34]:
#Level of water stress: freshwater withdrawal as a proportion of available freshwater resources
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Level%20of%20water%20stress%20freshwater%20withdrawal%20as%20a%20proportion%20of%20available%20freshwater%20resources/FAO%20AQUASTAT%20Dissemination%20System.csv"
level_of_water_stress = pd.read_csv(url)

# Limpezia del dataset
cols_to_drop = level_of_water_stress.columns.drop(["Area","Year","Value"])
level_of_water_stress.drop(columns=cols_to_drop, inplace=True)
level_of_water_stress.rename(columns={"Area": "Country Name","Value":"Level of water stress: freshwater withdrawal as a proportion of available freshwater resources"}, inplace=True)

level_of_water_stress['Country Name'] = level_of_water_stress['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Level of water stress: freshwater withdrawal as a proportion of available freshwater resources"] = level_of_water_stress

# Verificar la forma
level_of_water_stress.head()

,Country Name,Year,Level of water stress: freshwater withdrawal as a proportion of available freshwater resources
0,Afghanistan,1975,28.887689
1,Afghanistan,1976,32.354909
2,Afghanistan,1977,35.822129
3,Afghanistan,1978,39.289349
4,Afghanistan,1979,42.756569


### Literacy rate, adult total (% of people ages 15 and above)

In [35]:
# Literacy rate, adult total (% of people ages 15 and above)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Literacy%20rate,%20adult%20total%20(%25%20of%20people%20ages%2015%20and%20above)/literacy-rate-by-country-2025.csv"
literacy_rate = pd.read_csv(url)

# Limpezia del dataset
cols_to_drop = literacy_rate.columns.drop(["country","LiteracyRate_DataYear_year_MultiYear","LiteracyRate_TotalPopulation_pct_MultiYear"])
literacy_rate.drop(columns=cols_to_drop, inplace=True)
literacy_rate.rename(columns={"country": "Country Name","LiteracyRate_TotalPopulation_pct_MultiYear":"Literacy rate, adult total (% of people ages 15 and above)",
                              "LiteracyRate_DataYear_year_MultiYear":"Year"}, inplace=True)


literacy_rate['Year'] = literacy_rate['Year'].astype(str).str.replace(r"\..*", "", regex=True)

literacy_rate['Country Name'] = literacy_rate['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Literacy rate, adult total (% of people ages 15 and above)"] = literacy_rate

# Verificar la forma
literacy_rate.head()

,Country Name,"Literacy rate, adult total (% of people ages 15 and above)",Year
0,Ukraine,100.0,2021
1,Uzbekistan,100.0,2022
2,"Korea, Dem. People's Rep.",100.0,2015
3,Kazakhstan,100.0,2020
4,Azerbaijan,100.0,2023


### Mammal species, threatened

In [36]:
# Mammal species, threatened
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Mammal%20species,%20threatened/API_EN.MAM.THRD.NO_DS2_en_csv_v2_6141.csv"
mammal_species_threatened = pd.read_csv(url)

# Limpezia del dataset
mammal_species_threatened.drop(columns=["Country Code","Indicator Name","Indicator Code"], inplace=True)

mammal_species_threatened = pd.melt(mammal_species_threatened,
                                     id_vars=["Country Name"],
                                     var_name="Year",
                                     value_name="Mammal species, threatened")

mammal_species_threatened['Country Name'] = mammal_species_threatened['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Mammal species, threatened"] = mammal_species_threatened

# Verificar la forma
mammal_species_threatened.head()

,Country Name,Year,"Mammal species, threatened"
0,Aruba,1960,NaN
1,Africa Eastern and Southern,1960,NaN
2,Afghanistan,1960,NaN
3,Africa Western and Central,1960,NaN
4,Angola,1960,NaN


### Mortality rate, under-5 (per 1,000 live births)

In [37]:
# Mortality rate, under-5 (per 1,000 live births)
url = "https://ourworldindata.org/grapher/child-mortality.csv?v=1&csvType=full&useColumnShortNames=false"
mortality_rate = pd.read_csv(url, storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Limpezia del dataset
mortality_rate.drop(columns=["Code"], inplace=True)
mortality_rate.rename(columns={"Entity": "Country Name","Child mortality rate":"Mortality rate, under-5 (per 1,000 live births)"}, inplace=True)

mortality_rate['Country Name'] = mortality_rate['Country Name'].replace(country_name_map)

# Les données sont en %
mortality_rate['Mortality rate, under-5 (per 1,000 live births)'] = mortality_rate['Mortality rate, under-5 (per 1,000 live births)']*10

# Add the dataset to the list
new_datas["Mortality rate, under-5 (per 1,000 live births)"] = mortality_rate

# Verificar la forma
mortality_rate.head()

,Country Name,Year,"Mortality rate, under-5 (per 1,000 live births)"
0,Afghanistan,1957,371.3
1,Afghanistan,1958,365.2
2,Afghanistan,1959,359.5
3,Afghanistan,1960,353.2
4,Afghanistan,1961,347.6


###Population density (people per sq. km of land area)

In [38]:
# Population density (people per sq. km of land area)
url = "https://ourworldindata.org/grapher/population-density.csv?v=1&csvType=full&useColumnShortNames=false"
population_density = pd.read_csv(url, storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Limpezia del dataset
population_density.drop(columns=["Code"], inplace=True)
population_density.rename(columns={"Entity": "Country Name","Population density":"Population density (people per sq. km of land area)"}, inplace=True)

population_density['Country Name'] = population_density['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Population density (people per sq. km of land area)"] = population_density

# Verificar la forma
population_density.head()

,Country Name,Year,Population density (people per sq. km of land area)
0,Afghanistan,-10000,0.022595
1,Afghanistan,-9000,0.031285
2,Afghanistan,-8000,0.043318
3,Afghanistan,-7000,0.059979
4,Afghanistan,-6000,0.083047


### Poverty headcount ratio at national poverty lines (% of population)

In [39]:
# Poverty headcount ratio at national poverty lines (% of population)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Poverty%20headcount%20ratio%20at%20national%20poverty%20lines%20(%25%20of%20population)/dataHub_poverty.csv"
poverty_headcount_ratio = pd.read_csv(url)

# Limpezia del dataset
poverty_headcount_ratio.drop(columns=["Country Code"], inplace=True)
poverty_headcount_ratio.rename(columns={"Country": "Country Name","Value":"Poverty headcount ratio at national poverty lines (% of population)"}, inplace=True)

poverty_headcount_ratio['Country Name'] = poverty_headcount_ratio['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Poverty headcount ratio at national poverty lines (% of population)"] = poverty_headcount_ratio

# Verificar la forma
poverty_headcount_ratio.head()

,Country Name,Year,Poverty headcount ratio at national poverty lines (% of population)
0,Afghanistan,2007,36.3
1,Afghanistan,2011,35.8
2,Albania,2002,25.4
3,Albania,2005,18.5
4,Albania,2008,12.4


###Prevalence of undernourishment (% of population)

In [40]:
# Prevalence of undernourishment (% of population)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Prevalence%20of%20undernourishment%20(%25%20of%20population)/DF_SDG_2_1_1.csv"
prevalence_of_undernourishment = pd.read_csv(url, storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Limpezia del dataset
prevalence_of_undernourishment = prevalence_of_undernourishment[(prevalence_of_undernourishment["UNIT_MEASURE_DESC"] == "Percent") & (prevalence_of_undernourishment["AREA"] != "Micronesia")]

cols_to_drop = prevalence_of_undernourishment.columns.drop(["AREA","TIME_PERIOD","OBS_VALUE"])
prevalence_of_undernourishment.drop(columns=cols_to_drop, inplace=True)
prevalence_of_undernourishment.rename(columns={"AREA": "Country Name","TIME_PERIOD":"Year","OBS_VALUE":"Prevalence of undernourishment (% of population)"}, inplace=True)

prevalence_of_undernourishment['Country Name'] = prevalence_of_undernourishment['Country Name'].replace(country_name_map)

# Corriger les valeurs de type <2.5
prevalence_of_undernourishment["Prevalence of undernourishment (% of population)"] = prevalence_of_undernourishment["Prevalence of undernourishment (% of population)"].replace(r"<\s*([0-9.]+)", r"\1", regex=True)
prevalence_of_undernourishment["Prevalence of undernourishment (% of population)"] = prevalence_of_undernourishment["Prevalence of undernourishment (% of population)"].astype(float)

# Add the dataset to the list
new_datas["Prevalence of undernourishment (% of population)"] = prevalence_of_undernourishment

# Verificar la forma
prevalence_of_undernourishment.head()

,Country Name,Year,Prevalence of undernourishment (% of population)
0,World,2001,12.8
1,World,2002,13.0
2,World,2003,12.7
3,World,2004,12.6
4,World,2005,12.0


### Proportion of bodies of water with good ambient water quality

In [41]:
#Proportion of bodies of water with good ambient water quality
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Proportion%20of%20bodies%20of%20water%20with%20good%20ambient%20water%20quality/UNWATER.csv"
proportion_of_bodies_of_water = pd.read_csv(url, encoding='latin-1')

# Limpezia del dataset
proportion_of_bodies_of_water.drop(columns=["Rivers","Open water bodies","Groundwater"], inplace=True)
proportion_of_bodies_of_water.rename(columns={"Country (or area), region, world": "Country Name"," Period" : "Year", "Overall":"Proportion of bodies of water with good ambient water quality"}, inplace=True)

proportion_of_bodies_of_water['Country Name'] = proportion_of_bodies_of_water['Country Name'].replace(country_name_map)
proportion_of_bodies_of_water['Year'] = proportion_of_bodies_of_water['Year'].astype(str).str.replace(r"[0-9]+-", "", regex=True)

# Add the dataset to the list
new_datas["Proportion of bodies of water with good ambient water quality"] = proportion_of_bodies_of_water

# Verificar la forma
proportion_of_bodies_of_water.head()

,Country Name,Year,Proportion of bodies of water with good ambient water quality
0,Albania,2023,39.06
1,Andorra,2023,68.00
2,Angola,2023,75.25
3,Antigua and Barbuda,2023,0.00
4,Argentina,2023,62.20


###Proportion of seats held by women in national parliaments (%)

In [42]:
#Proportion of seats held by women in national parliaments (%)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Proportion%20of%20seats%20held%20by%20women%20in%20national%20parliaments%20(%25)/Goal5.csv"
proportion_of_seats_held_by_women = pd.read_csv(url, encoding='latin-1')

# Limpezia del dataset
proportion_of_seats_held_by_women = proportion_of_seats_held_by_women[(proportion_of_seats_held_by_women["SeriesDescription"] == "Proportion of seats held by women in national parliaments (% of total number of seats)")]

cols_to_drop = proportion_of_seats_held_by_women.columns.drop(["GeoAreaName","TimePeriod","Value"])
proportion_of_seats_held_by_women.drop(columns=cols_to_drop, inplace=True)
proportion_of_seats_held_by_women.rename(columns={"GeoAreaName": "Country Name","TimePeriod" : "Year", "Value":"Proportion of seats held by women in national parliaments (%)"}, inplace=True)

proportion_of_seats_held_by_women['Country Name'] = proportion_of_seats_held_by_women['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Proportion of seats held by women in national parliaments (%)"] = proportion_of_seats_held_by_women

# Verificar la forma
proportion_of_seats_held_by_women

,Country Name,Year,Proportion of seats held by women in national parliaments (%)
481,World,2000,13.34
482,World,2001,13.79
483,World,2002,14.21
484,World,2003,15.12
485,World,2004,15.21
...,...,...,...
6425,Zambia,2021,16.77
6426,Zambia,2022,15.06
6427,Zambia,2023,14.97
6428,Zambia,2024,14.97


###Ratio of female to male labor force participation rate (%) (modeled ILO estimate)

In [43]:
# Ratio of female to male labor force participation rate (%) (modeled ILO estimate)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Labor%20force%20participation%20rate,%20total%20(%25%20of%20total%20population%20ages%2015-64)%20(modeled%20ILO%20estimate)/EAP_DWAP_SEX_AGE_RT_A-20250915T2011.csv"
ratio_of_female_to_male_labor = pd.read_csv(url,low_memory=False)

# Limpezia del dataset
cols_to_keep = ["ref_area.label","time","obs_value"]

male = ratio_of_female_to_male_labor[(ratio_of_female_to_male_labor['classif1.label'] == 'Age (Youth, adults): 15-64') & (ratio_of_female_to_male_labor['sex.label'] == 'Male')][cols_to_keep]
female = ratio_of_female_to_male_labor[(ratio_of_female_to_male_labor['classif1.label'] == 'Age (Youth, adults): 15-64') & (ratio_of_female_to_male_labor['sex.label'] == 'Female')][cols_to_keep]

ratio_of_female_to_male_labor = pd.merge(male,
                 female,
                 on=["ref_area.label","time"],
                 how="outer",
                 suffixes=("_male", "_female"))

# Calcul de l'ind
ratio_of_female_to_male_labor["Ratio of female to male labor force participation rate (%) (modeled ILO estimate)"] = ratio_of_female_to_male_labor["obs_value_female"]/ratio_of_female_to_male_labor['obs_value_male']*100

# On enlève les colonnes inutiles
ratio_of_female_to_male_labor.drop(columns=["obs_value_male","obs_value_female"], inplace=True)
ratio_of_female_to_male_labor.rename(columns={"ref_area.label": "Country Name","time":"Year"}, inplace=True)

ratio_of_female_to_male_labor['Country Name'] = ratio_of_female_to_male_labor['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Ratio of female to male labor force participation rate (%) (modeled ILO estimate)"] = ratio_of_female_to_male_labor

# Verificar la forma
ratio_of_female_to_male_labor.head()

,Country Name,Year,Ratio of female to male labor force participation rate (%) (modeled ILO estimate)
0,Afghanistan,1979,7.413393
1,Afghanistan,2008,53.069419
2,Afghanistan,2012,20.555576
3,Afghanistan,2014,33.258635
4,Afghanistan,2017,28.853448


###Renewable energy consumption (% of total final energy consumption)

In [44]:
#Renewable energy consumption (% of total final energy consumption)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Renewable%20energy%20consumption%20(%25%20of%20total%20final%20energy%20consumption)/sdg_out_000015369_i23_en.csv"
renewable_energy_consumption = pd.read_csv(url, encoding='latin-1')

# Limpezia del dataset

renewable_energy_consumption.drop(columns=["Country_ID"," Indicator_Id"," Indicator_E"], inplace=True)
renewable_energy_consumption.rename(columns={" Country_E":"Country Name"," Period":"Year"," Value":"Renewable energy consumption (% of total final energy consumption)"}, inplace=True)

renewable_energy_consumption['Country Name'] = renewable_energy_consumption['Country Name'].replace(country_name_map)
renewable_energy_consumption['Year'] = renewable_energy_consumption['Year'].astype(str)

#2e dataset
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Renewable%20energy%20consumption%20(%25%20of%20total%20final%20energy%20consumption)/Europa_Renewable_energy.csv"
renewable_energy_consumption_2 = pd.read_csv(url)

# Limpezia del dataset
renewable_energy_consumption_2.rename(columns={"Unnamed: 0":"Country Name"," Period":"Year","7.2.1 - Renewable energy share in the total final energy consumption (%) - EG_FEC_RNEW":"Renewable energy consumption (% of total final energy consumption)"}, inplace=True)
renewable_energy_consumption_2 = renewable_energy_consumption_2.melt(id_vars=["Country Name"], var_name="Year", value_name="Renewable energy consumption (% of total final energy consumption)")

renewable_energy_consumption_2['Country Name'] = renewable_energy_consumption_2['Country Name'].replace(country_name_map)
renewable_energy_consumption_2['Renewable energy consumption (% of total final energy consumption)'] = renewable_energy_consumption_2['Renewable energy consumption (% of total final energy consumption)'].astype(float)

renewable_energy_consumption = pd.merge(renewable_energy_consumption,
                 renewable_energy_consumption_2,
                 on=["Country Name","Year"],
                 how="outer",
                 suffixes=("", "_new"))

renewable_energy_consumption['Renewable energy consumption (% of total final energy consumption)']= renewable_energy_consumption['Renewable energy consumption (% of total final energy consumption)'].fillna(renewable_energy_consumption['Renewable energy consumption (% of total final energy consumption)_new'])
renewable_energy_consumption.drop(columns=["Renewable energy consumption (% of total final energy consumption)_new"], inplace=True)

# Add the dataset to the list
new_datas["Renewable energy consumption (% of total final energy consumption)"] = renewable_energy_consumption

# Verificar la forma
renewable_energy_consumption.head()

,Country Name,Year,Renewable energy consumption (% of total final energy consumption)
0,Albania,2004,29.620
1,Albania,2005,31.367
2,Albania,2006,32.070
3,Albania,2007,32.657
4,Albania,2008,32.448


###Research and development expenditure (% of GDP)

In [45]:
# Research and development expenditure (% of GDP)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Research%20and%20development%20expenditure%20(%25%20of%20GDP)/OECD_Research.csv"
research_and_development_expenditure = pd.read_csv(url)

# Limpezia del dataset
cols_to_drop = research_and_development_expenditure.columns.drop(["Zone de référence","TIME_PERIOD","OBS_VALUE"])
research_and_development_expenditure.drop(columns=cols_to_drop, inplace=True)
research_and_development_expenditure.rename(columns={"Zone de référence":"Country Name","TIME_PERIOD":"Year","OBS_VALUE":"Research and development expenditure (% of GDP)"}, inplace=True)

research_and_development_expenditure['Country Name'] = research_and_development_expenditure['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Research and development expenditure (% of GDP)"] = research_and_development_expenditure

# Verificar la forma
research_and_development_expenditure.head()

,Country Name,Year,Research and development expenditure (% of GDP)
0,Australia,1981,0.901285
1,Australia,1984,1.021601
2,Australia,1986,1.176432
3,Australia,1987,1.135337
4,Australia,1988,1.161209


### School enrollment, primary (% gross)

In [46]:
# School enrollment, primary (% gross)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/School%20enrollment,%20primary%20(%25%20gross)/UNESCO_school_enrollment_primary.csv"
school_enrollment_primary = pd.read_csv(url)

# Limpezia del dataset

school_enrollment_primary.drop(columns=["indicatorId","qualifier","qualifier","magnitude"], inplace=True)
school_enrollment_primary.rename(columns={"geoUnit": "Country Name","year":"Year","value":"School enrollment, primary (% gross)"}, inplace=True)

school_enrollment_primary['Country Name'] = school_enrollment_primary['Country Name'].replace(country_name_map)
school_enrollment_primary['School enrollment, primary (% gross)'] = school_enrollment_primary['School enrollment, primary (% gross)'].str.replace(",",".").astype(float)

# Add the dataset to the list
new_datas["School enrollment, primary (% gross)"] = school_enrollment_primary

# Verificar la forma
school_enrollment_primary.head()

,Country Name,Year,"School enrollment, primary (% gross)"
0,Aruba,2001,110.349670
1,Aruba,2002,115.669449
2,Aruba,2003,115.985001
3,Aruba,2004,117.217171
4,Aruba,2005,116.929039


### School enrollment, primary and secondary (gross), gender parity index (GPI)

In [47]:
# School enrollment, primary and secondary (gross), gender parity index (GPI)
url = "https://ourworldindata.org/grapher/net-enrollment-rate-primary-gender-parity-index-gpi.csv?v=1&csvType=full&useColumnShortNames=false"
school_enrollment = pd.read_csv(url, storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Limpezia del dataset
school_enrollment.drop(columns=["Code"], inplace=True)
school_enrollment.rename(columns={"Entity": "Country Name","Net enrolment rates in primary education, adjusted gender parity index":"School enrollment, primary and secondary (gross), gender parity index (GPI)"}, inplace=True)

school_enrollment['Country Name'] = school_enrollment['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["School enrollment, primary and secondary (gross), gender parity index (GPI)"] = school_enrollment

# Verificar la forma
school_enrollment.head()

,Country Name,Year,"School enrollment, primary and secondary (gross), gender parity index (GPI)"
0,Afghanistan,1974,0.18900
1,Afghanistan,1993,0.33804
2,Albania,2003,0.97474
3,Albania,2004,0.99089
4,Albania,2008,1.00642


### Tree Cover Loss

In [48]:
# Tree Cover Loss
url = "https://ourworldindata.org/grapher/tree-cover-loss.csv?v=1&csvType=full&useColumnShortNames=false"
tree_cover_loss = pd.read_csv(url, storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Limpezia del dataset
tree_cover_loss.drop(columns=["Code"], inplace=True)
tree_cover_loss.rename(columns={"Entity": "Country Name","Tree cover loss driven by all causes":"Tree Cover Loss"}, inplace=True)

tree_cover_loss['Country Name'] = tree_cover_loss['Country Name'].replace(country_name_map)

# Add the dataset to the list
new_datas["Tree Cover Loss"] = tree_cover_loss

# Verificar la forma
tree_cover_loss.head()

,Country Name,Year,Tree Cover Loss
0,Afghanistan,2001,70.865295
1,Afghanistan,2002,152.580280
2,Afghanistan,2003,221.000640
3,Afghanistan,2004,183.536850
4,Afghanistan,2005,211.495970


### Unmet need for contraception (% of married women ages 15-49)

Voir si j'ai pas fait n'importe quoi

In [49]:
# Unmet need for contraception (% of married women ages 15-49)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Unmet%20need%20for%20contraception%20(%25%20of%20married%20women%20ages%2015-49)/undesa_pd_2024_wcu_country_data_survey-based.csv"
unmet_need_for_contraception = pd.read_csv(url)

# Limpezia del dataset
unmet_need_for_contraception = unmet_need_for_contraception[unmet_need_for_contraception["Age group"] == "15-49"]
cols_to_drop = unmet_need_for_contraception.columns.drop(["Country or area","Survey\nend year","Unmet need\ntotal"])
unmet_need_for_contraception.drop(columns=cols_to_drop, inplace=True)
unmet_need_for_contraception.rename(columns={"Country or area": "Country Name","Survey\nend year":"Year","Unmet need\ntotal":"Unmet need for contraception (% of married women ages 15-49)"}, inplace=True)

unmet_need_for_contraception['Country Name'] = unmet_need_for_contraception['Country Name'].replace(country_name_map)

unmet_need_for_contraception["Unmet need for contraception (% of married women ages 15-49)"] = unmet_need_for_contraception["Unmet need for contraception (% of married women ages 15-49)"].str.replace("..","NaN")
unmet_need_for_contraception["Unmet need for contraception (% of married women ages 15-49)"] = unmet_need_for_contraception["Unmet need for contraception (% of married women ages 15-49)"].str.replace(",",".")
unmet_need_for_contraception["Unmet need for contraception (% of married women ages 15-49)"] = unmet_need_for_contraception["Unmet need for contraception (% of married women ages 15-49)"].astype(float)

# Duplicated
unmet_need_for_contraception = unmet_need_for_contraception.groupby(['Country Name', 'Year'])["Unmet need for contraception (% of married women ages 15-49)"].mean().reset_index()

# Unmet need for contraception (% of married women ages 15-49)
url = "https://github.com/Nolhandm/Trabajo-de-Maestria/raw/refs/heads/main/Objetivo%201%20-%20Integracion%20de%20datos/Datasets/Unmet%20need%20for%20contraception%20(%25%20of%20married%20women%20ages%2015-49)/datahub.csv"
unmet_need_for_contraception_2 = pd.read_csv(url)

# Limpezia del datase
unmet_need_for_contraception_2.drop(columns=['Country Code'], inplace=True)
unmet_need_for_contraception_2.rename(columns={"Entity": "Country Name","Survey\nend year":"Year","Value":"Unmet need for contraception (% of married women ages 15-49)"}, inplace=True)

unmet_need_for_contraception_2['Country Name'] = unmet_need_for_contraception_2['Country Name'].replace(country_name_map)

unmet_need_for_contraception = pd.merge(unmet_need_for_contraception,
                 unmet_need_for_contraception_2,
                 on=["Country Name","Year"],
                 how="outer",
                 suffixes=("", "_new"))

unmet_need_for_contraception['Unmet need for contraception (% of married women ages 15-49)']= unmet_need_for_contraception['Unmet need for contraception (% of married women ages 15-49)'].fillna(unmet_need_for_contraception['Unmet need for contraception (% of married women ages 15-49)_new'])
unmet_need_for_contraception.drop(columns=["Unmet need for contraception (% of married women ages 15-49)_new"], inplace=True)

# Add the dataset to the list
new_datas["Unmet need for contraception (% of married women ages 15-49)"] = unmet_need_for_contraception

# Verificar la forma
unmet_need_for_contraception.head()

,Country Name,Year,Unmet need for contraception (% of married women ages 15-49)
0,Afghanistan,2008,NaN
1,Afghanistan,2010,NaN
2,Afghanistan,2011,NaN
3,Afghanistan,2015,24.5
4,Afghanistan,2016,24.5


## Ajout des nouvelles données

In [50]:
# Nombre de valeurs manquantes avant le début de l'ajout

missing, per = missing_per()
print(f"Sur l'ensemble du Dataset il manque {missing} valeurs sur {nb_val_dataset} soit {per}% des valeurs")

Sur l'ensemble du Dataset il manque 689231 valeurs sur 1224080 soit 56.31% des valeurs


In [51]:
# Colonnes des années
year_columns = ESG_df.columns.drop(["Country Name","Indicator Name"])
for ind in ESG_df['Indicator Name'].unique():

  print(f"----------\n{ind}")
  start_missing, start_per = missing_per_ind(ind)
  print(f"Pour cet indicateur {nb_val_ind-start_missing} valeurs sont présente sur {nb_val_ind} soit {np.round(100-start_per,2)}% des valeurs")

  if ind in new_datas.keys():

    max,min,mean,q,country,year,valeur_ref,valeur_comp = diff_moy_rel(ind)
    print(f"Les valeurs du nouveau dataset présentent une différence relative moyenne de {np.round(mean,2)}%, avec un maximum de {np.round(max,2)}%, un minimum de {np.round(min,2)}% et un 80e centile de {np.round(q,2)}%")
    print(f"Le maximum concerne {country} en {year} avec une valeur de référence de {valeur_ref} et une valeur comparée de {valeur_comp}")

    # Pivot du Dataframe des nouvelles données pour avoir le même format qu'ESG_df
    pivot_df = new_datas[ind].pivot(
        index="Country Name",
        columns="Year",
        values=ind
    )
    pivot_df.columns = pivot_df.columns.astype(str)

    # On garde uniquement les lignes de l'indicateur
    indicator_mask = ESG_df["Indicator Name"] == ind

    # Select the relevant part of ESG_df and set Country Name as index
    target_df = ESG_df.loc[indicator_mask].set_index("Country Name")

    # Update the target_df with values from pivot_gdp, only filling NaN values
    target_df.update(pivot_df, overwrite=False)

    # Copy the updated values back to the original ESG_df
    ESG_df.loc[indicator_mask,year_columns] = target_df[year_columns].values

    not_found_countries = list(set(pivot_df.index.unique()) - set(list_countries_complete))
    if len(not_found_countries) > 0:
      print("Attention les données concernant les pays suivants n'ont pas pu être ajoutées", not_found_countries)
    end_missing, end_per = missing_per_ind(ind)
    print(f"Pour cet indicateur {nb_val_ind-end_missing} valeurs sont présente sur {nb_val_ind} soit {np.round(100-end_per,2)}% des valeurs, {start_missing-end_missing} valeurs ont été ajoutées")
  else:
    print("Pas de nouvelles données")

----------
Access to clean fuels and technologies for cooking (% of population)
Pour cet indicateur 4347 valeurs sont présente sur 13910 soit 31.25% des valeurs
Les valeurs du nouveau dataset présentent une différence relative moyenne de 2.26%, avec un maximum de 239.56%, un minimum de 0.0% et un 80e centile de 1.15%
Le maximum concerne Palau en 2022 avec une valeur de référence de 29.45 et une valeur comparée de 100.0
Attention les données concernant les pays suivants n'ont pas pu être ajoutées ['World']
Pour cet indicateur 4536 valeurs sont présente sur 13910 soit 32.61% des valeurs, 189 valeurs ont été ajoutées
----------
Access to electricity (% of population)
Pour cet indicateur 6437 valeurs sont présente sur 13910 soit 46.28% des valeurs
Pas de nouvelles données
----------
Adjusted savings, natural resources depletion (% of GNI)
Pour cet indicateur 7415 valeurs sont présente sur 13910 soit 53.31% des valeurs
Pas de nouvelles données
----------
Adjusted savings, net forest depleti

In [52]:
ESG_df['Indicator Name'].nunique()

88

In [53]:
# Nombre de valeurs manquantes après l'ajout

new_missing, new_per = missing_per()
print(f"Sur l'ensemble du Dataset il manque à présent {new_missing} valeurs sur {nb_val_dataset} soit {new_per}% des valeurs, {missing-new_missing} valeurs ({np.round(per-new_per,2)}%) ont été ajoutées")

Sur l'ensemble du Dataset il manque à présent 674578 valeurs sur 1224080 soit 55.11% des valeurs, 14653 valeurs (1.2%) ont été ajoutées


# Dataframe to CSV

In [54]:
# Dataframe to CSV
ESG_df.to_csv('ESG_df_Completed.csv', index=False)

In [55]:
ESG_df

,Country Name,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Cameroon,Access to clean fuels and technologies for coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.8000,23.6000,24.3000,25.3000,26.4000,27.4000,28.6000,29.4000,30.7000,NaN
1,Cameroon,Access to electricity (% of population),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,58.6000,59.8000,60.9000,62.2000,63.2000,64.3000,65.4000,71.0000,72.0000,NaN
2,Cameroon,"Adjusted savings, natural resources depletion ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.4405,5.2334,5.8511,5.6127,5.0737,4.0527,4.9676,NaN,NaN,NaN
3,Cameroon,"Adjusted savings, net forest depletion (% of GNI)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.6341,3.7438,3.8193,3.0194,2.6998,2.7600,2.6397,NaN,NaN,NaN
4,Cameroon,Agricultural land (% of land area),NaN,15.8871,16.014,16.1198,16.2467,16.3737,16.5006,16.6487,...,20.6258,20.6258,20.6258,20.6258,20.6258,20.6258,20.6258,20.6258,20.9400,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18827,American Samoa,Total greenhouse gas emissions per capita excl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.1589,0.1608,0.1628,0.1650,0.1693,0.1708,0.1727,0.1758,0.1789,NaN
18828,American Samoa,Tree Cover Loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18829,American Samoa,"Unemployment, total (% of total labor force) (...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18830,American Samoa,Unmet need for contraception (% of married wom...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
